# Python Intro - Final Exercise


In [3]:
import pyodbc
import pandas as pd
import numpy as np

In [4]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')


In [5]:
### Get All tables
students = pd.read_sql("SELECT * FROM Students;",conn) 
Classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn) 
Courses = pd.read_sql("SELECT * FROM Courses;",conn) 
Departments = pd.read_sql("SELECT * FROM Departments;",conn) 
Teachers = pd.read_sql("SELECT * FROM Teachers;",conn) 

In [6]:
students.head()

,StudentId,FirstName,LastName,Gender
0,1,Alexandria,Heller,F
1,2,Cassie,Liska,F
2,3,Hayley,Frank,F
3,4,Maxwell,Kotch,M
4,5,Dylan,Boehm,M


In [7]:
Classrooms.head()

,CourseId,StudentId,degree
0,1,202,92.031070
1,1,12,81.175916
2,1,232,82.396587
3,1,123,62.420449
4,1,274,72.569900


In [8]:
Courses.head()

,CourseId,CourseName,DepartmentID,TeacherId
0,1,English Begginers,1,15
1,2,Advanced English,1,15
2,3,Proffesional English,1,15
3,4,Mathematics 101,2,2
4,5,Matematics Advanced,2,13


In [9]:
Departments.head()

,DepartmentId,DepartmentName
0,1,English
1,2,Science
2,3,Arts
3,4,Sport


# Questions
 
 
## Q1. Count the number of students on each department

In [10]:
df_Classroom_Courses = pd.merge(Classrooms, Courses, how='inner', left_on='CourseId', right_on='CourseId')
df_Departments = pd.merge(df_Classroom_Courses,Departments,how='inner', left_on='DepartmentID', right_on='DepartmentId')
result1 = df_Departments.groupby('DepartmentName').StudentId.nunique()
result1 = result1.reset_index()
result1.columns = ['Department Name','Student Count']
result1


,Department Name,Student Count
0,Arts,137
1,English,84
2,Science,200
3,Sport,81


## Q2. How many students have each course of the English department and the total number of students in the department?

In [11]:
result2 = df_Departments[df_Departments.DepartmentName =='English'].groupby('CourseName').agg(['count'])
result2 = result2.iloc[:, 0:1]
result2.columns = ['Student Count'] 
print(result2)
print("--------------------")
totalSum = df_Departments[df_Departments.DepartmentName =='English'].groupby('DepartmentName').nunique().iloc[:, 0:2]
print('Total Student On English Dept' + "    " + totalSum['StudentId'].apply(str)[0])


                      Student Count
CourseName                         
Advanced English                 24
English Begginers                32
Proffesional English             38
--------------------
Total Student On English Dept    84


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [12]:
dfClassScience = df_Departments[df_Departments.DepartmentName == 'Science'].groupby('CourseName').agg(['count'])
Big_Class_Count = 0
Small_Class_Count = 0
columnDatavalues = 0
for (columnName, columnData) in dfClassScience.iteritems():
    #print('Column Contents : ', columnData.values)
    result3 = columnData.values
    break 

for x in result3:
    if x >= 22 :
        Big_Class_Count = Big_Class_Count + 1
    else:
        Small_Class_Count = Small_Class_Count + 1


print("Big Class  :   " + str(Big_Class_Count))
print("Small Class:   " + str(Small_Class_Count))


Big Class  :   10
Small Class:   6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [13]:
df_students = students
result4 = df_students.groupby('Gender').agg(['count'])
result4 = result4.iloc[:, 0:1]
result4.columns = ['Student Count'] 

print(result4)

        Student Count
Gender               
F                 165
M                 115


## Q5. For which courses the percentage of male/female students is over 70%?

In [14]:
df_Class_Courses = pd.merge(Classrooms, Courses, how='inner', left_on='CourseId', right_on='CourseId')
df_Student_Courses = pd.merge(df_Class_Courses,students,how='inner', left_on='StudentId', right_on='StudentId')
df_AllStudent = df_Class_Courses.groupby(['CourseId']).agg(['count']).reset_index().iloc[:, 0:2]
result5 = df_Student_Courses.groupby(['CourseId','CourseName','Gender']).agg(['count']).reset_index().iloc[:, 0:4]
result5 = pd.merge(result5, df_AllStudent, how='inner', left_on='CourseId', right_on='CourseId')
result5['StudentPercent'] = (result5['StudentId_x']/ result5['StudentId_y'])*100.0
result5 = result5[result5['StudentPercent'] > 70.0]
result5 = result5.drop(result5.columns[3:5], axis=1)
print(result5.to_string(index=False))


CourseId CourseName Gender StudentPercent
                                         
      22  Sculpture      F      70.833333
      29      Tenis      F      72.222222


C:\Users\sharabi\anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## Q6. For each department, how many students passed with a grades over 80?

In [15]:
df_Cl_Co = pd.merge(Classrooms, Courses, how='inner', left_on='CourseId', right_on='CourseId')
total_student = df_Cl_Co.groupby(['DepartmentID']).nunique().iloc[:, 0:2]
total_student = total_student["StudentId"]
#print(total_student)
df_Cl_Co = df_Cl_Co[df_Cl_Co['degree'] > 80.0] 
sudent_over_80 = df_Cl_Co.groupby(['DepartmentID']).nunique()
sudent_over_80 = sudent_over_80["StudentId"]
#print(sudent_over_80)
result6 = pd.merge(Departments, sudent_over_80, how='inner', left_on='DepartmentId', right_on='DepartmentID')
result6 = pd.merge(result6, total_student, how='inner', left_on='DepartmentId', right_on='DepartmentID')
result6['Student_80_pct'] = (result6['StudentId_x'] / result6['StudentId_y'])*100.0
result6.columns = ['Department Id ','Department Name','Student Count Over 80','Total Student','Student Percent Over 80'] 
print(result6.to_string(index=False))


 Department Id  Department Name  Student Count Over 80  Total Student  Student Percent Over 80
              1         English                     40             84                47.619048
              2         Science                    122            200                61.000000
              3            Arts                     61            137                44.525547
              4           Sport                     39             81                48.148148


## Q7. For each department, how many students passed with a grades under 60?

In [16]:
df_Cl_Co7 = pd.merge(Classrooms, Courses, how='inner', left_on='CourseId', right_on='CourseId')
total_student7 = df_Cl_Co7.groupby(['DepartmentID']).nunique().iloc[:, 0:2]
total_student7 = total_student7["StudentId"]
#print(total_student)
df_Cl_Co7 = df_Cl_Co7[df_Cl_Co7['degree'] < 60.0] 
sudent_under_60 = df_Cl_Co7.groupby(['DepartmentID']).nunique()
sudent_under_60 = sudent_under_60["StudentId"]
#print(sudent_over_80)
result7 = pd.merge(Departments, sudent_under_60, how='inner', left_on='DepartmentId', right_on='DepartmentID')
result7 = pd.merge(result7, total_student7, how='inner', left_on='DepartmentId', right_on='DepartmentID')
result7['Student_60_pct'] = (result7['StudentId_x'] / result7['StudentId_y'])*100.0
result7.columns = ['Department Id ','Department Name','Student Count Under 60','Total Student','Student Percent Over 60'] 
print(result7.to_string(index=False))

 Department Id  Department Name  Student Count Under 60  Total Student  Student Percent Over 60
              1         English                       2             84                 2.380952
              2         Science                       7            200                 3.500000
              3            Arts                       3            137                 2.189781
              4           Sport                       1             81                 1.234568


## Q8. Rate the teachers by their average student's grades (in descending order).

In [17]:
df_Cl_Co8 = pd.merge(Classrooms, Courses, how='inner', left_on='CourseId', right_on='CourseId')
tmp_result8 = df_Cl_Co8[['degree','TeacherId']]
tmp_result8 = tmp_result8.groupby('TeacherId').mean()
result8 = pd.merge(tmp_result8,Teachers,how='inner', left_on='TeacherId', right_on='TeacherId')
result8 =  result8[['LastName','FirstName','degree']]
result8 = result8.sort_values(by='degree', ascending=False)
result8


,LastName,FirstName,degree
19,Hess,Darick,81.432703
2,Baron,Jordyn,80.729494
3,Metcalf,Heather,80.440242
17,Healy,Zachary,79.365867
0,Monzingo,Kaylea,79.365537
15,Bland,Daniel,78.908281
21,Price,Keegan,78.814631
12,Mckittrick,Conor,78.699492
20,Romkee,Maxwell,78.673509
16,Beaty,Terran,78.495479


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [18]:
dfCD9 = pd.merge(Courses, Departments, how='left', left_on='DepartmentID', right_on='DepartmentId')
dfCD9C = pd.merge(dfCD9, Classrooms, how='left', left_on='CourseId', right_on='CourseId')
dfCD9C = pd.merge(dfCD9C, Teachers, how='left', left_on='TeacherId', right_on='TeacherId')
dfCD9C = dfCD9C.groupby(['CourseId','CourseName','DepartmentName','FirstName','LastName']).StudentId.nunique()
dfCD9C = dfCD9C.reset_index()
dfCD9C.columns = ['Course Id ','Course Name','Department Name','First Name','Last Name', 'Student Number']
dfCD9C


,Course Id,Course Name,Department Name,First Name,Last Name,Student Number
0,1,English Begginers,English,Conor,Mckittrick,32
1,2,Advanced English,English,Conor,Mckittrick,24
2,3,Proffesional English,English,Conor,Mckittrick,38
3,4,Mathematics 101,Science,Amanda,Dworkin,18
4,5,Matematics Advanced,Science,Alexander,Watts,10
5,6,Algebra,Science,Alexander,Watts,12
6,7,Geometry,Science,Alexander,Watts,13
7,8,Chemistry,Science,Jeffrey,Batzel,23
8,9,Physics,Science,Alexander,Watts,12
9,10,Biology,Science,Jordyn,Baron,36


## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [19]:
df10 = pd.merge(students, Classrooms, how='left', left_on='StudentId', right_on='StudentId')
df10 = pd.merge(df10, Courses, how='left', left_on='CourseId', right_on='CourseId')

df_Count_Courses = df10.groupby(['StudentId', 'FirstName','LastName']).CourseId.nunique()
df_Student = df_Count_Courses.reset_index()
df_Student.columns = ['StudentId','First Name','Last Name', 'Count Courses']

df_English = df10[df10['DepartmentID']==1]
df_Mean_English = df_English.groupby(['StudentId'])['degree'].mean().reset_index()
df_Mean_English.columns = ['StudentId','EnglishDegree']
df_Student = pd.merge(df_Student, df_Mean_English, how='left', left_on='StudentId', right_on='StudentId')
####### Science Mean
df_Science = df10[df10['DepartmentID']==2]
df_Mean_Science = df_Science.groupby(['StudentId'])['degree'].mean().reset_index()
df_Mean_Science.columns = ['StudentId','ScienceDegree']
df_Student = pd.merge(df_Student, df_Mean_Science, how='left', left_on='StudentId', right_on='StudentId')
####### Arts Mean
df_Arts = df10[df10['DepartmentID']==3]
df_Mean_Arts = df_Arts.groupby(['StudentId'])['degree'].mean().reset_index()
df_Mean_Arts.columns = ['StudentId','ArtsDegree']
df_Student = pd.merge(df_Student, df_Mean_Arts, how='left', left_on='StudentId', right_on='StudentId')
####### Sport Mean
df_Sport = df10[df10['DepartmentID']==4]
df_Mean_Sport = df_Sport.groupby(['StudentId'])['degree'].mean().reset_index()
df_Mean_Sport.columns = ['StudentId','SporteDegree']
df_Student = pd.merge(df_Student, df_Mean_Sport, how='left', left_on='StudentId', right_on='StudentId')
####### General Mean
df_Mean_Avarage = df10.groupby(['StudentId'])['degree'].mean().reset_index()
df_Mean_Avarage
df_Mean_Avarage.columns = ['StudentId','AvarageDegree']
df_Student = pd.merge(df_Student, df_Mean_Avarage, how='left', left_on='StudentId', right_on='StudentId')
df_Student




,StudentId,First Name,Last Name,Count Courses,EnglishDegree,ScienceDegree,ArtsDegree,SporteDegree,AvarageDegree
0,1,Alexandria,Heller,4,NaN,79.557508,86.677535,NaN,81.337515
1,2,Cassie,Liska,5,NaN,78.867115,79.542738,NaN,79.137364
2,3,Hayley,Frank,3,83.972245,76.204427,NaN,NaN,78.793700
3,4,Maxwell,Kotch,3,70.929801,74.157066,69.356144,NaN,71.481003
4,5,Dylan,Boehm,3,NaN,100.000000,64.494845,65.661106,76.718650
...,...,...,...,...,...,...,...,...,...
275,276,Dylan,Bannister,3,NaN,76.784735,NaN,NaN,76.784735
276,277,Bryan,Medina,6,NaN,81.298837,78.849330,NaN,80.482335
277,278,Sydney,Blas,3,NaN,72.578017,NaN,76.749775,75.359189
278,279,Samantha,Berend,1,NaN,NaN,NaN,NaN,NaN
